In [53]:
'''
Periodic aware routing usually cause deadline constraint violation 
compared with shortest path.

'''

'\nPeriodic aware routing usually cause deadline constraint violation \ncompared with shortest path.\n\n'

In [54]:
# CPO_PATH = '/home/ubuntu/Cplex/cpoptimizer/bin/x86-64_linux/cpoptimizer'
CPO_PATH = '/opt/ibm/ILOG/CPLEX_Studio221/cpoptimizer/bin/x86-64_linux/cpoptimizer'
# CPO_PATH = '/home/cc/tool/CPLEX_Studio221/cpoptimizer/bin/x86-64_linux/cpoptimizer'


In [55]:
import pandas as pd
import numpy as np
import z3
# from tqdm.notebook import tqdm
# from tqdm.autonotebook import tqdm
from tqdm import tqdm
# blah blah your code errored
tqdm._instances.clear()

In [56]:
from docplex.cp.model import *

In [57]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60
NUM_FLOW = 128
# DATA_NAME = "orion2"
DATA_NAME = "harmonic0"
TOPO_NAME = "1"

# task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
# task['period'] = task['period'] * 1_000
# network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)

task = pd.read_csv("../../data/stream/stream_%s_%s.csv" % (18, 0))
network = pd.read_csv("../../data/stream/stream_topology.csv")

# task = pd.read_csv("../../data/utilization/utilization_5_34.csv")
# network = pd.read_csv("../../data/utilization/utilization_topology.csv")

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [58]:
task

,id,src,dst,size,period,deadline,jitter
0,0,14,[10],4,4000,418,418
1,1,12,[9],2,8000,230,230
2,2,14,[10],4,1000,418,418
3,3,9,[11],4,1000,418,418
4,4,11,[14],5,8000,250,250
5,5,13,[11],2,8000,604,604
6,6,12,[9],2,8000,230,230
7,7,14,[11],4,2000,326,326
8,8,11,[15],5,2000,300,300
9,9,9,[15],5,8000,350,350


## 1. Model

Network model:

$\left\langle\left[v_{a}, v_{b}\right] . s,\left[v_{a}, v_{b}\right] . d,\left[v_{a}, v_{b}\right] . m t,\left[v_{a}, v_{b}\right] . c\right\rangle$

In [59]:
net_var = {}
for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['d'] = row['t_proc']
    net_var[row['link']]['c'] = row['q_num']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

Task model：

$\left\langle f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot \phi, f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot T, f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot L\right\rangle$

Use shortest route in this paper.

In [60]:
task_var = {}

In [61]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

## 1. Optimum routing (Logic-Based Benders Decomposition)

In [62]:
s = z3.Optimize()
s.set("timeout", 5 * 60*60*1000)

In [63]:
task_path = {}
for i in list(task['id']):
    task_path.setdefault(i, {})
    for link in net_var:
        task_path[i][link] = z3.Int('path_' + str(i) + '+' + str(link))
        s.add(
            0 <= task_path[i][link],
            task_path[i][link] <= 1
        )

In [64]:
for i in task_path:
    s.add(
        z3.Sum([task_path[i][xa] for xa in net_var if eval(xa)[1] == eval(task.loc[i, 'dst'])[0]])
        == 1
    )
    s.add(
        z3.Sum([task_path[i][xa] for xa in net_var if eval(xa)[0] == task.loc[i, 'src']])
        == 1
    )
        
    ### Add constraint that only 2 ES in path
    for link in net_var:
        if (eval(link)[0] in ES_set and eval(link)[0] != task.loc[i, 'src']) \
        or (eval(link)[1] in ES_set and eval(link)[1] != eval(task.loc[i, 'dst'])[0]):
            s.add(
                task_path[i][link] == 0
            )
for i in task_path:
    for v in SW_set:
        s.add(
            z3.Sum([task_path[i][xa] for xa in net_var if eval(xa)[1] == v])
            ==
            z3.Sum([task_path[i][ay] for ay in net_var if eval(ay)[0] == v])
        )
        
        ## Add constaint that only pass SW once
        s.add(
            z3.Sum([task_path[i][xa] for xa in net_var if eval(xa)[1] == v])
            <= 1
        )
        s.add(
            z3.Sum([task_path[i][ay] for ay in net_var if eval(ay)[0] == v])
            <= 1
        )
        
        
# print(s.check())

### 1.1 Shortest Paths

$$\min \sum_{\substack{i \in \mathcal{F} \\(a, b) \in \mathcal{E}}} \rho_{i, a, b}$$

In [65]:
s.minimize(
    z3.Sum(
        [task_path[i][link] for i in task_path for link in net_var]
    )
)

In [66]:
res = s.check()
result = s.model()

### 1.2 Load Balancing

$$\forall(a, b) \in \mathcal{E}: \sum_{i \in \mathcal{F}} \rho_{i, a, b} \cdot \frac{L_i}{T_i} \leqslant v$$

In [67]:
# s.minimize(
#     z3.Sum(
#         [task_path[i][link] * int(task.loc[i, 'size'] * 8 / task.loc[i, 'period']) for i in task_path for link in net_var]
#     )

# )

In [68]:
# res = s.check()
# result = s.model()

In [69]:
# for i in task_path:
#     print([link for link in task_path[i] if result[task_path[i][link]].as_long() == 1])

### 1.3 Period Matching

In [70]:
# period_path = {}

In [71]:
# for link in net_var:
#     period_path.setdefault(link, {})
#     for p in task['period'].unique():
#         period_path[link][p] = z3.Int('x_' + str(p) + '+' + str(link))
#         s.add(
#             0 <= period_path[link][p],
#             period_path[link][p] <= 1
#         )
#         for p_p in task['period'].unique():
#             if p_p > p:
#                 period_path[link][(p, p_p)] = z3.Int('y_' + str((p, p_p)) + '+' + str(link))
#                 s.add(
#                     0 <= period_path[link][(p, p_p)],
#                     period_path[link][(p, p_p)] <= 1
#                 )

\begin{aligned}
&\forall(a, b) \in \mathcal{E}, \forall p \in\{1, \ldots,|\mathcal{P}|\}: x_{p, a, b} \leqslant \sum_{i \in \mathcal{F}, T_i=P_p} \rho_{i, a, b} \\
&\forall(a, b) \in \mathcal{E}, \forall p \in\{1, \ldots,|\mathcal{P}|\}, \forall i \in \mathcal{F}, T_i=P_p: \rho_{i, a, b} \leqslant x_{p, a, b}
\end{aligned}

In [72]:
# for link in net_var:
#     for p in task['period'].unique():
#         s.add(
#             period_path[link][p] <= z3.Sum([task_path[i][link] for i in list(task[task['period'] == p]['id'])])
#         )

In [73]:
# for link in net_var:
#     for p in task['period'].unique():
#         for i in list(task[task['period'] == p]['id']):
#             s.add(
#                 task_path[i][link] <= period_path[link][p]
#             )

\begin{aligned}
&\forall(a, b) \in \mathcal{E}, \forall p, p^{\prime} \in\{1, \ldots,|\mathcal{P}|\}, p<p^{\prime}: \\
&y_{p, p^{\prime}, a, b} \leqslant x_{p, a, b} \\
&y_{p, p^{\prime}, a, b} \leqslant x_{p^{\prime}, a, b} \\
&x_{p, a, b}+x_{p^{\prime}, a, b} \leqslant y_{p, p^{\prime}, a, b}+1
\end{aligned}

In [74]:
# for link in net_var:
#     for p, p_p in [(p, p_p) for p in task['period'].unique() for p_p in task['period'].unique() if p_p > p]:
#         s.add(
#             period_path[link][(p, p_p)] <= period_path[link][p],
#             period_path[link][(p, p_p)] <= period_path[link][p_p],
#             period_path[link][p] + period_path[link][p_p] <= period_path[link][(p, p_p)] + 1
#         )
        

In [75]:
# s.minimize(
#     z3.Sum(
#         [np.lcm(p, p_p) / np.gcd(p, p_p) * period_path[link][(p, p_p)]
#          for link in net_var 
#          for p, p_p in [(p, p_p) for p in task['period'].unique() for p_p in task['period'].unique() if p_p > p]]
#     )

# )

In [76]:
# res = s.check()
# print(res)
# result = s.model()

In [77]:
for i in task_path:
    task_path[i] = [link for link in task_path[i] if result[task_path[i][link]].as_long() == 1]

## 2. Constraints

In [78]:
s = CpoModel()

In [79]:
def path_sort(start, route):
    route = [x for x in route]
    if len(route) == 0:
        return []
    for i, link in enumerate(route):
        if eval(link)[0] == start:
            route.pop(i)
            return [link] + path_sort(eval(link)[1], route)
    return []

In [80]:
## Assume task is strictly periodic
task_var = {}
for i, row in task.iterrows():
    task_var.setdefault(i, {})
    route = path_sort(task.loc[i, 'src'], task_path[i])
    for _i, link in enumerate(route):
        link = str(link)
        task_var[i].setdefault(link, {})
        task_var[i][link]['phi'] = []
        task_var[i][link]['T'] = int(row['period'])
        task_var[i][link]['p'] = s.integer_var()
        s.add(
            task_var[i][link]['p'] >= 0,
            task_var[i][link]['p'] <= 7
        )
        L = int((row['size'] * 8 / net_var[str(link)]['s']))
        for k in range(int(LCM / task_var[i][link]['T'])):
            task_var[i][link]['phi'].append(
                s.interval_var(
                    size = L,
                    start = [k * task_var[i][link]['T'], (k+1) * task_var[i][link]['T'] - L]
                )
            )
        
        

### 2.1 General constraints

Frame Constraint

\begin{equation}
\forall i \in \mathcal{F}, \forall(a, b) \in \mathcal{E}, \forall k \in\left\{1, \ldots, \frac{H P}{T_i}-1\right\}: \operatorname{StartAtStart}\left(I_{i, k-1}^{a, b}, I_{i, k}^{a, b}, T_i\right)
\end{equation}

In [81]:
for i in task_var:
    for link in task_var[i]:
        for k in range(int(LCM / task_var[i][link]['T']) - 1):
            s.add(
                s.start_at_start(task_var[i][link]['phi'][k], task_var[i][link]['phi'][k + 1], task_var[i][link]['T'])
            )

\begin{equation}
\begin{array}{l}
\forall i \in \mathcal{F}, \forall(a, b) \in \mathcal{E}, \forall k \in\left\{1, \ldots, \frac{H P}{T_i}-1\right\} \\
\quad: \text { PresenceOf }\left(I_{i, k-1}^{a, b}\right) \Leftrightarrow \text { PresenceOf }\left(I_{i, k}^{a, b}\right)
\end{array}
\end{equation}

In [82]:
## This constraint is ignored for Logic-based benders decomposition

\begin{equation}
\forall(a, b) \in \mathcal{E}: \operatorname{NoOverlap}\left(\bigcup_{k \in\left\{\frac{i \in \mathcal{F}}{0, \ldots, \frac{\mu_T}{T_i}-1}\right\}}\left\{I^{a, b}\right\}\right)
\end{equation}

In [83]:
for link in net_var:
    no_overlap_links = [task_var[i][link_i]['phi'] for i in task_var for link_i in task_var[i] if link_i == link]
    no_overlap_links = [x for y in no_overlap_links for x in y]
    if no_overlap_links:
        s.add(
            s.no_overlap(no_overlap_links)
        )

\begin{equation}
\forall i \in \mathcal{F}, \forall(x, a),(a, b) \in \mathcal{E}: \operatorname{StartBeforeStart}\left(I_{i, 0}^{x, a}, I_{i, 0}^{a, b}, L_i+d_{x, a}+d_a+\delta\right)
\end{equation}

In [84]:
for i in task_var:
    for hop, link in enumerate(list(task_var[i].keys())[:-1]):
        next_link = list(task_var[i].keys())[hop + 1]
        s.add(
            s.end_before_start(task_var[i][link]['phi'][0], task_var[i][next_link]['phi'][0], net_var[link]['d'])
        )

In [85]:
## Deadline constraint
for i in task_var:
    in_link = list(task_var[i].keys())[0]
    out_link = list(task_var[i].keys())[-1]
    s.add(
        s.end_of(task_var[i][out_link]['phi'][0]) - s.start_of(task_var[i][in_link]['phi'][0]) <= task.loc[i, 'deadline'] - net_var[out_link]['d']
    )

### 2.2 IEEE 802.1 Qbv constraint

\begin{equation}
\begin{aligned}
\forall i, j \in \mathcal{F}, i<j, \forall(a, b) \in \mathcal{E}, \forall \alpha \in\left\{0, \ldots, \frac{\operatorname{lcm}\left\{T_i, T_j\right\}}{T_i}-1\right\}, \forall \beta \\
\in &\left\{0, \ldots, \frac{\operatorname{lcm}\left\{T_i, T_j\right\}}{T_j}-1\right\} \\
: &\left(\text { PresenceOf }\left(I_{i, 0}^{a, b}\right) \wedge \operatorname{PresenceOf}\left(I_{j, 0}^{a, b}\right) \wedge \lambda_{i, a, b}\right.\\
=&\left.\lambda_{j, a, b}\right) \Rightarrow\left(\left(\operatorname{StartOf}\left(I_{i, 0}^{a, b}\right) \leqslant \sum_{(y, a) \in \mathcal{E}} \operatorname{PresenceOf}\left(I_{j, 0}^{y, a}\right) \cdot\left(\operatorname{StartOf}\left(I_{j, 0}^{y, a}\right)\right.\right.\right.\\
&\left.\left.+L_j+d_{y, a}+\beta \cdot T_j-\alpha \cdot T_i-\delta\right)\right) \\
& \vee\left(\operatorname{StartOf}\left(I_{j, 0}^{a, b}\right) \leqslant \sum_{(x, a) \in \mathcal{E}} \operatorname{PresenceOf}\left(I_{i, 0}^{x, a}\right) \cdot\left(\operatorname{StartOf}\left(I_{i, 0}^{x, a}\right)+L_i\right.\right.\\
&\left.\left.\left.+d_{x, a}+\alpha \cdot T_i-\beta \cdot T_j-\delta\right)\right)\right)
\end{aligned}
\end{equation}

In [86]:
## Stream / Frame isolation

for i, j in [(i,j) for i in task_var for j in task_var if i < j]:
    path_i = list(task_var[i].keys())
    path_j = list(task_var[j].keys())
    i_period, j_period = int(task.loc[i]['period']), int(task.loc[j]['period'])
    for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b) 
                        for _x, i_a_b in enumerate(path_i) 
                        for _y, j_a_b in enumerate(path_j) 
                        if i_a_b == j_a_b and _x != 0 and _y != 0]:
        # print(x_a, y_a, a_b)
        lcm = np.lcm(i_period, j_period)
        for a, b in [(a, b)
                        for a in range(0, int(lcm / i_period))
                        for b in range(0, int(lcm / j_period))]:
            s.add(
                s.logical_or(
                        task_var[i][a_b]['p'] != task_var[j][a_b]['p'],
                        s.logical_or(
                        s.start_of(task_var[i][a_b]['phi'][0]) + a * i_period < s.start_of(task_var[j][y_a]['phi'][0]) + b * j_period + net_var[a_b]['d'],
                        s.start_of(task_var[j][a_b]['phi'][0]) + b * j_period < s.start_of(task_var[i][x_a]['phi'][0]) + a * i_period + net_var[a_b]['d'],
                        )
                    )
                # s.if_then(
                #     task_var[i][a_b]['p'] == task_var[j][a_b]['p'],
                #     ## A SIMPLE ERROR REQUIRES TO BE MERGED INTO GITREPO
                #     s.logical_or(
                #         s.start_of(task_var[i][a_b]['phi'][0]) + a * i_period < s.start_of(task_var[j][y_a]['phi'][0]) + b * j_period + net_var[a_b]['d'],
                #         s.start_of(task_var[j][a_b]['phi'][0]) + b * j_period < s.start_of(task_var[i][x_a]['phi'][0]) + a * i_period + net_var[a_b]['d'],
                #     ) == True
                # )
            )

FIFO constraint - Designed by Chuanyu Jul 2 2022

In [87]:
start_time = time.time()
result = s.solve(
        agent='local',
        # execfile='/home/ubuntu/Cplex/cpoptimizer/bin/x86-64_linux/cpoptimizer',
        execfile=CPO_PATH,
        LogVerbosity='Quiet',
        # SearchType = 'DepthFirst',
        Workers = 1,
        TimeLimit=time_out)
end_time = time.time()
res = result.get_solve_status()

In [88]:
res

'Infeasible'

In [ ]:
if res == 'Unknown' or res == 'SearchStoppedByLimit':
    pass
if res == 'Feasible':
    pass
if res == 'Infeasible':
    pass

In [ ]:
# for i in task_var:
#     print('[Task %ad] ------------------------------'%i)
#     for link in task_var[i]:
#         pass
#         # print(result.get_value(task_var[i][link]['phi'][0]))
#         # print(result.get_value(task_var[i][link]['phi'][1]))
#         # print(result.get_value(task_var[i][link]['p']))
#         # try:
#         #     print(result.get_value(task_var[i][link]['p']))
#         # except:
#         #     print(0)

## Output schedule

In [ ]:
## GCL
GCL = []
for i in task_var:
    for e in task_var[i]:
        start = result.get_value(task_var[i][e]['phi'][0]).start
        end = result.get_value(task_var[i][e]['phi'][0]).end
        queue = result.get_value(task_var[i][e]['p'])
        last = end
        queue = i
        t = task_var[i][e]['T']
        for k in range(int(LCM / t)):
            GCL.append(
                [eval(e), queue, (start + k * t) * macrotick, (end + k * t) * macrotick, LCM * macrotick]
            )

In [ ]:
## Offset
OFFSET = []
for i in task_var:
    offset = result.get_value(list(task_var[i].values())[0]['phi'][0]).start
    OFFSET.append(
        [i, 0, (list(task_var[i].values())[0]['T'] - offset) * macrotick]
    )    

In [ ]:
ROUTE = []
for i, row in task.iterrows():
    route = list(task_var[i].keys())
    for h, v in enumerate(route):
        ROUTE.append(
            [i, v]
        )
# with open('RTNS16-route-CBS-16.txt', 'w') as f:
#     f.write(str(route))

In [ ]:
QUEUE = []
for i in task_var:
    for e in task_var[i]:
        queue = result.get_value(task_var[i][e]['p'])
        queue = i
        QUEUE.append([i, 0, eval(e), queue])
        # queue[i][e] = result[task_var[i][e]['p']]
# with open('RTNS16-queue-CBS-16.txt', 'w') as f:
#     f.write(str(queue))

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("CIE2021-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("CIE2021-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("CIE2021-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("CIE2021-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [ ]:
result.get_solve_time()

In [ ]:
## In bytes -> to Megabytes
result.get_solver_infos()["PeakMemoryUsage"] / 1024 / 1024